In [18]:
#read in demo_data.csv and store as JSON
import csv
import json

#read in csv file
with open('demo_data.csv', 'r') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

#write to json file
with open('demo_data.json', 'w') as f:
    json.dump(rows, f, indent=2)

In [19]:
#read in json file into pandas
import pandas as pd
df = pd.read_json('demo_data.json')

In [25]:
import pickle
#load model
model = pickle.load(open('xgb_model.pkl', 'rb'))

In [20]:
#make a df for each unique time stamp
df=df[df['time']=='2024-05-21T19:00']

In [21]:
#calculate wind shear on df
import numpy as np
def calculate_wind_shear(speed1, speed2, dir1, dir2):
    # Convert wind directions from degrees to radians
    dir1_rad = np.radians(dir1)
    dir2_rad = np.radians(dir2)

    # Calculate the wind vector components at the first altitude
    u1 = speed1 * np.sin(dir1_rad)
    v1 = speed1 * np.cos(dir1_rad)

    # Calculate the wind vector components at the second altitude
    u2 = speed2 * np.sin(dir2_rad)
    v2 = speed2 * np.cos(dir2_rad)

    # Calculate the wind shear components
    shear_u = u2 - u1
    shear_v = v2 - v1

    # Calculate the magnitude of the wind shear vector
    shear_magnitude = np.sqrt(shear_u**2 + shear_v**2)
    
    return shear_magnitude

df['wind_shear'] = df.apply(lambda x: calculate_wind_shear(x['wind_speed_10m'], x['wind_speed_100m'], x['wind_direction_10m'], x['wind_direction_100m']), axis=1)

In [22]:
df.columns

Index(['ï»¿location_id', 'time', 'temperature_2m', 'relative_humidity_2m',
       'rain', 'pressure_msl', 'surface_pressure', 'wind_speed_10m',
       'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m',
       'soil_temperature_0_to_7cm', 'wind_shear'],
      dtype='object')

In [23]:
features=['temperature_2m', 'relative_humidity_2m',
       'rain', 'pressure_msl', 'surface_pressure', 'wind_speed_10m',
       'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m',
       'soil_temperature_0_to_7cm', 'wind_shear']

In [24]:
X=df[features]

In [26]:
#Make a dataframe that has Time | County | Prediction that is empty
df_st= pd.DataFrame(columns=['Time', 'County', 'Prediction'])

In [34]:
#for each record in the df, make a prediction and append to df_st. proba off features X, df[Prediction] off model
max_time= df['time'].max()

for county in df['county_name'].unique():
    X_county= X[df['county_name']==county]
    y_pred= model.predict(X_county)
    y_proba= model.predict_proba(X_county)
    df_st= df_st.append({'Time': max_time, 'County': county, 'Prediction': y_proba}, ignore_index=True)

KeyError: 'county_name'